## Sequential Search
This notebook finds the optimal $3 \times 3$ and $4 \times 4$ matrices related to the paper "On the Structure of Bad Science Matrices" through an exhaustive, sequential search.


First import all necessary libraries:

In [1]:
import torch
from itertools import combinations, islice, product
from tqdm import tqdm
import numpy as np
from scipy import special
import timeit

Helper functions used to generate the candidate rows for the optimal matrix. These are explained in detail in the $\texttt{construct_candidate_rows}$ notebook.

In [2]:
# Generate binary vectors
def vectors(n, binary=False):
    if binary:
        return torch.tensor(list(product([0, 1], repeat=n)), dtype=torch.float32).T
    return torch.tensor(list(product([-1, 1], repeat=n)), dtype=torch.float32).T

# Currently there are rows that appear twice up to a sign change, and this function keeps only one of them.
def fix_row_signs(tensor):
    # Iterate over each row
    normalized_tensor = []
    for row in tensor:
        # Find the first non-zero element for sign determination
        first_non_zero_index = (row != 0).nonzero(as_tuple=True)[0][0]
        sign = torch.sign(row[first_non_zero_index])
        normalized_row = row * sign
        normalized_tensor.append(normalized_row)
    normalized_tensor = torch.stack(normalized_tensor)
    # Now return the unique ones.
    return torch.unique(normalized_tensor, dim=0)

# Generates all possible rows appearing in the optimal matrix. Now it's a matter of checking all row combinations.
def candidate_rows(n):
  # Each row represents a subset
  binary_choices = vectors(2**n, binary=True).T

  # Sums over chosen subset to yield all candidate, unnormalized rows
  rows = torch.matmul(binary_choices, vectors(n).T)

  # Compute their norms
  row_norms = torch.norm(rows, dim=1, keepdim=True)

  # Normalize rows
  normalized_rows_with_nans = rows/row_norms

  # During normalization, some choices add up to 0, so after normalizing we obtain all nan values
  # We thus slice the tensor to remove nan rows
  normalized_rows = normalized_rows_with_nans[~torch.all(normalized_rows_with_nans.isnan(), dim = 1)]

  # Remove redundant rows (no need to try them twice!)
  unique_normalized_rows = torch.unique(normalized_rows, dim=0)

  # We can also remove rows that are equal up to a row-wise sign change.
  final_rows = fix_row_signs(unique_normalized_rows)

  return final_rows

The choice of $n$ sets whether we want to check $3 \times 3$ or $4 \times 4$ matrices. Note that the $n = 4$ case can be prohibitively time consuming to check on Google colab due to the time limitations. We recommend running it locally.

The following code cell also defines the beta function of a matrix, and the $\texttt{_run(combination, rows)}$ function builds a matrix from a given index $n$-tuple, chosen from the rows of the array $\texttt{rows}$.  


In [10]:
n = 3

# Generate candidate rows
rows = candidate_rows(n)

# Binary vectors needed to compute the beta value of a matrix
binary_vectors = vectors(n)[:, :2**(n-1)]

# Beta function
def beta(A, binary_vectors):
    B = torch.matmul(A, binary_vectors)
    return torch.mean(torch.max(torch.abs(B), dim=0).values)

# For matrix from combinations
def _run(combination, rows):
    A = rows[torch.tensor(combination)]
    value_A = beta(A, binary_vectors)
    return value_A, A

Having defined those, we create a generator to parse through all possible combinations ($2300$ for $n = 3$ and $65536$ for $n = 4$), and parse through each combination, keeping track of the maximum beta value, stored in the $\texttt{beta_max}$ variable.

In [11]:
elements = torch.tensor(range(rows.shape[0]))

def generate_combinations(elements, r):
    for combination in combinations(elements, r):
        yield combination

combinations_generator = generate_combinations(elements, n)

start = timeit.timeit()

beta_max = 0

best_A = None

total = special.binom(rows.shape[0], rows.shape[1]).astype(int)

print(f"Checking {total} matrices")

for combination in tqdm(combinations_generator, total = total):

    A = rows[torch.tensor(combination)]

    beta_candidate = torch.mean(torch.max(torch.abs(A @ binary_vectors), dim=0).values)

    if beta_candidate > beta_max:
        beta_max, best_A = beta_candidate, A

print()
print(f"Beta = {beta_max}")
print(f"Best A is {best_A.numpy()}")

end = timeit.timeit()

print(f"Sequential took {np.abs(end-start)} seconds")

Checking 2300 matrices


100%|██████████| 2300/2300 [00:00<00:00, 13495.15it/s]


Beta = 1.5731321573257446
Best A is [[ 0.          0.70710677 -0.70710677]
 [ 0.57735026 -0.57735026 -0.57735026]
 [ 0.57735026  0.57735026  0.57735026]]
Sequential took 0.006619795999995404 seconds
